In [2]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense

from util import util
from util import WavFileParts
from util.confusionMatrix import ConfusionMatrix


Using TensorFlow backend.


In [3]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = winDir

baseTargetDir = '/Users/etto/Desktop/pData'
baseTargetDir = 'E:/SAA_DATA/targetDir'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

NFFT = 1024

storageFolder = '../storedData/'
chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

In [4]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

## evalueer op wavFileParts

In [4]:
wfPts = readSoundChunksDynamic(chunksBaseDir + '.' + 'Studio' + '.soundChunks')

In [10]:
datasetDir = osDir + 'localizationRecordings/20171011'
fileDate = 170816
micNr = 4

### creeer spectros en sla ze op

In [31]:
allSpectros = np.array([])
allClasses = np.array([])

startTime = datetime.now()
print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

for wfPt in wfPts: #type: WavFilePart
    if not 'Gunshot' in wfPt.getSoundType():
        filename = datasetDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, wfPt.fileNr, micNr)
        fs, signal = wv.read(filename)
        
        classNr = classes.index(wfPt.getSoundType().lower())
        for soundChunk in wfPt.getSoundChunks(micNr):
            startFrame = int(soundChunk[0] * fs)
            endFrame = int(soundChunk[1] * fs)

            sigChunk = signal[startFrame: endFrame]
            freq_array, segment_times, spectrogram = sig.spectrogram(x=sigChunk, fs=fs, nfft=NFFT, noverlap=0)
            if len(allSpectros) == 0:
                allSpectros = spectrogram.T
            else:
                allSpectros = np.append(allSpectros, spectrogram.T, axis=0)
            allClasses = np.append(allClasses, classNr * np.ones(len(segment_times)))

endTime = datetime.now()
print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
print('Time taken: '),
print(endTime - startTime)


filename = storageFolder + '20180717.hd5'
df = pd.DataFrame(allSpectros)
df.to_hdf(path_or_buf=filename, key='spectrosStudio')

df = pd.DataFrame(allClasses)
df.to_hdf(path_or_buf=filename, key='classesStudio')

Start: 12:44:37


C:\Users\Etto\Anaconda2\envs\py36DL\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)



End: 12:46:05
Time taken: 
0:01:28.084052
Ready


In [7]:
# Lees spectros weer in
filename = storageFolder + '20180717.hd5'
specDf = pd.read_hdf(path_or_buf=filename, key='spectrosStudio')
classesDf = pd.read_hdf(path_or_buf=filename, key='classesStudio')
print(np.shape(specDf))
print(np.shape(classesDf))

(143836, 513)
(143836, 1)


In [14]:
soundModel = load_model(storageFolder + 'prf20180714.hd5')
# predicted classes
predictions = soundModel.predict(specDf.values)
predClasses = predictions.argmax(axis=1)

matrix = ConfusionMatrix(classes)
for vals in zip(classesDf.values, predClasses):
    matrix.add(int(vals[0]), int(vals[1]), 1)
print(matrix.toString())
print()
print(matrix.toF1String())

                   m       v       e |  sens   acc
music          37600   22090    3905 |  0.59  0.82
voice              6   35926    1972 |  0.95  0.82
environment       11    1552   40774 |  0.96  0.95
--------------------------------------
prec            1.00    0.60    0.87

F1 overall: 0.80
F1 music: 0.74
F1 voice: 0.74
F1 environment: 0.92



In [16]:
predClasses

array([2, 2, 2, ..., 1, 1, 1], dtype=int64)

In [19]:
classesDf.values

array([[ 2.],
       [ 2.],
       [ 2.],
       ..., 
       [ 1.],
       [ 1.],
       [ 1.]])

In [22]:
a = list(zip(classesDf.values, predClasses))

In [23]:
vals = a[0]

In [24]:
vals

(array([ 2.]), 2)

In [25]:
int(vals[0])

2

In [27]:
int(vals[1])

2